In [130]:
import os       # Access the OS environment.
from langchain_community.document_loaders import WebBaseLoader      # Load the documentation.
from langchain_text_splitters import RecursiveCharacterTextSplitter  # Split the text data.
from langchain_community.embeddings import OllamaEmbeddings     # Convert the Text data into vectores.
from langchain_community.vectorstores import FAISS          # Going to save the vectores into vectore store data base.

# Load the Data

In [131]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()

In [132]:
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='\n\n\n\n\nGet started with LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith\u200bPythonTypeScriptpip install -U langsmithyarn add l

# Split the text data

In [133]:
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
text_split=documents.split_documents(docs)

In [134]:
text_split[0].page_content

'Get started with LangSmith | 🦜️🛠️ LangSmith'

In [135]:
text_split[1].page_content

"Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith\u200bPythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key\u200bTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment\u200bShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace\u200bTracing to LangSmith for LangChain usersThere is no need to use the LangSmith SDK directly if your application is built en

# Embedding - Convert Text data into Vectores

In [136]:
text_embeddings=OllamaEmbeddings(model="gemma2:2b")

# Vectore Store Data Base

In [137]:
text_db=FAISS.from_documents(text_split,text_embeddings)
retriever=text_db.as_retriever()

In [138]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000017BBB132E30>)

# Convert Retriever into Tools

In [139]:
from langchain.tools.retriever import create_retriever_tool

In [140]:
retriever_tool=create_retriever_tool(retriever,"langsmith-search","search asny information asbout langsmith")

In [141]:
retriever_tool.name

'langsmith-search'

# Tools Creation
## Arxiv
## Wikipedia

In [200]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun,DuckDuckGoSearchRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper

## Use the inbuilt tool of Wikipedia

In [143]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

In [144]:
wiki.name

'wikipedia'

## Use the inbuilt tool of Arxiv

In [145]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [146]:
arxiv.name

'arxiv'

# Use the unbuilt tool of Ducksearch

In [201]:
search=DuckDuckGoSearchRun()

## Combine the all tools

In [202]:
tools=[retriever_tool,wiki,arxiv,search]

In [203]:
tools

[Tool(name='langsmith-search', description='search asny information asbout langsmith', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000017BFBA90AF0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000017BBB132E30>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000017BFBA90B80>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000017BBB132E30>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Projects\\GenerativeAI\\GenAi\\lib\\

# Execute Tools and LLM with Agent Excutor
## Run all the tools with Agents and LLM model

In [204]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

True

## Load the Key

In [205]:
groq_api_key=os.getenv("GROQ_API_KEY")

## Create the LLM model by using Groq Fast API

In [206]:
llm=ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")

## Create the Prompt

In [207]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain import hub

In [208]:
prom=hub.pull("hwchase17/openai-functions-agent")

c:\Projects\GenerativeAI\GenAi\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [209]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","How may i help you!"),
        ("user","{input}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [210]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='How may i help you!')),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='messages')]

# Excute altogether 
## Agent

In [211]:
from langchain.agents import create_openai_tools_agent

In [212]:
agent=create_openai_tools_agent(llm=llm,tools=tools,prompt=prom)

In [213]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_has

## Agent Executer to run the Agent

In [214]:
from langchain.agents import AgentExecutor

In [215]:
agent_executor=AgentExecutor(agent=agent,tools=tools)

In [216]:
agent_executor

AgentExecutor(agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_h

In [217]:
query="tell me about langsmith"

In [218]:
agent_executor.invoke({"input":"Tell me about Machine Learning"})

c:\Projects\GenerativeAI\GenAi\lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
c:\Projects\GenerativeAI\GenAi\lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
c:\Projects\GenerativeAI\GenAi\lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be remove

{'input': 'Tell me about Machine Learning',
 'output': 'I think I can provide a direct answer without using any more tools. Machine learning has many applications in various fields, including healthcare, finance, marketing, transportation, and more. Some examples of machine learning applications include:\n\n* Predictive maintenance in manufacturing\n* Personalized medicine in healthcare\n* Credit risk assessment in finance\n* Recommendation systems in e-commerce\n* Self-driving cars in transportation\n\nThese are just a few examples, but the potential applications of machine learning are vast and continue to grow as the technology advances.'}

In [220]:
agent_executor.invoke({"input":"Tell me about Generative AI"})

c:\Projects\GenerativeAI\GenAi\lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
c:\Projects\GenerativeAI\GenAi\lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
c:\Projects\GenerativeAI\GenAi\lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be remove

{'input': 'Tell me about Generative AI',
 'output': "Based on the results provided by the tools, I'd like to provide a summary about Generative AI.\n\nGenerative AI, also known as Generative Artificial Intelligence, is a type of artificial intelligence that is capable of generating text, images, videos, or other data using generative models. These models are often trained on large datasets and can generate new data that is similar in style and quality to the training data. Generative AI is often used in applications such as language translation, image compression, and data augmentation.\n\nIn the context of natural language processing, generative AI can be used to generate text that is similar in style and tone to a given text. This can be useful for applications such as chatbots, language translation, and content generation.\n\nGenerative AI has many potential applications, including:\n\n1. Content generation: Generative AI can be used to generate high-quality content such as articles

In [221]:
agent_executor.invoke({"input":"about bengal incident"})

c:\Projects\GenerativeAI\GenAi\lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
c:\Projects\GenerativeAI\GenAi\lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


{'input': 'about bengal incident',
 'output': 'It seems that the tool call id "call_jwne" did not yield a relevant result for the query "Bengal incident". The result is actually a documentation for LangSmith, a natural language processing tool. This tool is not relevant to the current conversation about the Bengal incident.'}

In [222]:
agent_executor.invoke({"input":"tell me about Langsmith"})

c:\Projects\GenerativeAI\GenAi\lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
c:\Projects\GenerativeAI\GenAi\lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


{'input': 'tell me about Langsmith',
 'output': 'It seems like the tool call id "call_kpj4" yielded a lot of text about LangSmith. LangSmith is a platform for building production-grade large language model (LLM) applications. It allows users to closely monitor and evaluate their applications, making it easier to ship them quickly and with confidence. LangSmith provides a range of features and tools to help users build and deploy their LLM applications.\n\nBased on the results of the tool call, it appears that LangSmith is a comprehensive platform that offers a lot of functionality, including tracing, evaluation, and monitoring. It also seems that LangSmith can be used with other tools and platforms, such as LangChain and OpenAI.\n\nAs a helpful assistant, I can provide more information about LangSmith and its features if you\'re interested. Alternatively, I can also respond directly to your question about Langsmith without using any tools. Please let me know how I can assist you furthe